In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import requests
import re
import time
import random
import urllib3

urllib3.disable_warnings()

In [3]:
from src.summarize_text import SpeechSummaryProcessor

In [4]:
speech_to_text = SpeechSummaryProcessor(audio_path='data/audio_ogg.ogg')
text = speech_to_text.generate_speech_to_text()

Device cuda:0


In [5]:
text

"\n1: I'll be home later at 8 o'clock to pack my clothes\n\n2: At 9 o'clock I go to bed to write my essay and at 10 o'clock I go to bed"